<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Plan" data-toc-modified-id="Plan-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Plan</a></span></li><li><span><a href="#Code" data-toc-modified-id="Code-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Code</a></span><ul class="toc-item"><li><span><a href="#Import-modules-and-load-functions" data-toc-modified-id="Import-modules-and-load-functions-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Import modules and load functions</a></span></li><li><span><a href="#Get-dataverse-info" data-toc-modified-id="Get-dataverse-info-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Get dataverse info</a></span></li><li><span><a href="#Get-dataset-info" data-toc-modified-id="Get-dataset-info-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Get dataset info</a></span></li></ul></li></ul></div>

## Plan

For chosen dataverse:
    - Show dataverse info:
        - Show number of subdataverses, if any
        - Show whether or not the (sub)dataverse has a description or tagline
        - List metadatablocks of each (sub)dataverse
        - List the facets set for each (sub)dataverse
        - Verify that contact email address is valid for each (sub)dataverse
        (see https://medium.com/@arjunsinghy96/verify-emails-over-socks-proxy-using-python-5589cb75c405
        and https://github.com/Gardener-gg/email-verifier)
    - Show dataset info
        - Show number of datasets
        - Show date of first published dataset
        - Show date of most recently published or updated dataset
        - Show average age of datasets
        - Show average number of dataset versions
        - Metadata (of latest published version of each dataset):
            - Show average number of characters in the dataset descriptions
                - List datasets with fewer than a certain number of characters in their descriptions
            - List datasets with CC0 or Terms of Use metadata
                - Versus number of datasets with no CC0 or TOU metadata
                - List datasets with no CC0 or TOU metadata
            - Show number of files that have no description metadata
                - If a certain percentage of datasets have 1 or more files with no descriptions, list those datasets
            - Related publication metadata
                - Show number of datasets with related publication metadata
                    - List datasets with no related publication metadata
                - Show number of datasets with no PID in related publication metadata
                    - List datasets with no PID in related publication metadata
            - Show datasets that have no metadata for any non-citation metadatablocks enabled in the dataverse
        - Data
            - List count of each unique file format
            - Show number of datasets with no files
                - List datasets that have no files
            - Show number of datasets with 1 or more uningested tabular files
                - List datasets that contain 1 or more uningested tabular files
            - Show number of datasets with 1 or more restricted files
        - Contact emails
            - Get number of datasets that have a contact email address that's different email from the dataverse contact email address
            - Get unique list of contact email addresses and check if they're valid
            - Show any datasets that have no valid email addresses

## Code

### Import modules and load functions

In [1]:
from datetime import datetime, timezone
import numpy as np
import pandas as pd
import requests
import time


def improved_get(_dict, path, default=None):
    for key in path.split('.'):
        try:
            _dict = _dict[key]
        except KeyError:
            return default
    return _dict


def list_to_string(list):
    # Alphabetize list in case-insensitive way
    list = sorted(list, key=lambda s: s.casefold())
    # Change list to comma-separated string
    delimiter = ","
    string = delimiter.join(list)
    return string


def string_to_datetime(string):
    newDatetime = datetime.strptime(string, '%Y-%m-%dT%H:%M:%S%z')
    return newDatetime

# current_time = str.strftime('%Y.%m.%d_%H.%M.%S')
# current_time = datetime.strftime('%Y-%m-%dT%H:%M:%S%z')
# currentTime = datetime.now()
currentTime = datetime.now(timezone.utc)


### Get dataverse info

In [16]:
# Get dataverse server and alias from user - return error if there's no alias or if alias is the Root dataverse
# server = 'https://demo.dataverse.org'
# dataverseAlias = 'sefsef'
server = 'https://dataverse.harvard.edu'
dataverseAlias = 'IOJ'

repositoryMetadataBlocksApi = '%s/api/v1/metadatablocks' % (server)
response = requests.get(repositoryMetadataBlocksApi)
repositoryMetadataBlocks = response.json()

repositoryMetadataBlockNames = []
for repositoryMetadataBlock in repositoryMetadataBlocks['data']:
    repositoryMetadataBlockNames.append(repositoryMetadataBlock['name'])

In [17]:
# Get info from that dataverse: whether or not the dataverse has a description and/or tagline, metadatablocks enabled, facets enabled, validate contact email
dataverseInfoApi = '%s/api/dataverses/%s' % (server, dataverseAlias)
response = requests.get(dataverseInfoApi)
dataverseMetadata = response.json()

In [18]:
if dataverseMetadata['status'] == 'ERROR':
    print('No dataverse found. Is the dataverse published on Harvard Dataverse?')
elif dataverseMetadata['status'] == 'OK':
    dataverseDescription = improved_get(dataverseMetadata, 'data.description')
    if dataverseDescription is not None:
        print('Dataverse has a description')
    else:
        print('Dataverse has no description')

    tagline = improved_get(dataverseMetadata, 'data.theme.tagline')
    if tagline is not None:
        print('Dataverse has a tagline')
    else:
        print('Dataverse has no tagline')

#     contactEmails = []
#     for contact in dataverseMetadata['data']['dataverseContacts']:
#         contactEmails.append(contact['contactEmail'])
#     print(contactEmails)

    dataverseFacetsApi = '%s/api/dataverses/%s/facets' % (server, dataverseAlias)
    response = requests.get(dataverseFacetsApi)
    dataverseFacets = response.json()
    facets = []
    for facet in dataverseFacets['data']:
        facets.append(facet)
    print('Number of search facets used: %s' % (len(facets)))    

#     # See if dataverse inherits its metadatablocks from its parent dataverse
#     metadatablocksInheritedApi = '%s/api/dataverses/%s/metadatablocks/isRoot' % (server, dataverseAlias)
#     response = requests.get(metadatablocksInheritedApi)
#     metadatablocksInherited = response.json()
#     print(metadatablocksInherited)
    
    # Get list of metadatablocks enabled in the dataverse
    dataverseMetadatablocksList = []
    dataverseMetadatablocksApi = '%s/api/dataverses/%s/metadatablocks' % (server, dataverseAlias)
    response = requests.get(dataverseMetadatablocksApi)
    dataverseMetadatablocks = response.json()
    for dataverseMetadatablock in dataverseMetadatablocks['data']:
        dataverseMetadatablock = dataverseMetadatablock['name']
        dataverseMetadatablocksList.append(dataverseMetadatablock)
    print('Number of metadatablocks enabled (in addition to Citation): %s' % (len(dataverseMetadatablocksList) - 1))


Dataverse has a description
Dataverse has a tagline
Number of search facets used: 8
Number of metadatablocks enabled (in addition to Citation): 1


### Get dataset info

In [19]:
# Get PIDs of all published datasets and their files in that dataverse
getContentsApi = '%s/api/dataverses/%s/contents' % (server, dataverseAlias)
response = requests.get(getContentsApi)
dataverseContents = response.json()


In [20]:
datasetPIDs = []
for item in dataverseContents['data']:
    if item['type'] == 'dataset':
        datasetPID = item['persistentUrl'].replace('https://doi.org/', 'doi:')
        datasetPIDs.append(datasetPID)
print('Number of datasets: %s' % (len(datasetPIDs)))


Number of datasets: 65


Create a dataframe for dataset info: date of publication, the release date of the latest version, number of versions

_Getting this info can be slow. For example, getting the info of ~375 datasets might take 45 min_

In [21]:
# List of file types that Dataverse can convert to .tab files
uningestedFileTypes = ['application/x-rlang-transport', 'application/x-stata-13', 'application/x-spss-por',
                      'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 'text/csv', 'text/tsv',
                      'application/x-spss-sav', 'text/comma-separated-values', 'application/x-stata',
                      'application/x-stata-14']

rowList = []
datasetCount = 0
for datasetPID in datasetPIDs:
    getAllVersionsApi = '%s/api/datasets/:persistentId/versions?persistentId=%s' % (server, datasetPID)
    response = requests.get(getAllVersionsApi)
    datasetVersions = response.json()
    
    # Get only datasets with metadata (exclude responses with no values in 'data' key, e.g. deaccessioned datasets)
    if datasetVersions['status'] == 'OK' and len(datasetVersions['data']) > 0:

        # Get index location of first dataset version
        firstVersion = len(datasetVersions['data']) - 1

        publicationDate = string_to_datetime(datasetVersions['data'][firstVersion]['releaseTime'])
        latestReleaseDate = string_to_datetime(datasetVersions['data'][0]['releaseTime'])
        
        # Get age of dataset from today's date
        delta = currentTime - publicationDate
        ageOfDataset = delta.days
        
        # Get number of days since last update
        delta = currentTime - latestReleaseDate
        ageOfLastUpdate = delta.days
        
        # Get length of description text
        descriptionLength = 0
        for field in datasetVersions['data'][0]['metadataBlocks']['citation']['fields']:
            if field['typeName'] == 'dsDescription':
                for i in field['value']:
                    descriptionLength = descriptionLength + len(i['dsDescriptionValue']['value'])

        # See whether CC0 or Terms of Use metadata exists
        license = datasetVersions['data'][0]['license']
        try:
            termsOfUse = datasetVersions['data'][0]['termsOfUse']
            termsOfUse = True
        except KeyError:
            termsOfUse = False

        if license != 'CC0' and termsOfUse == False:
            termsExist = False
        else:
            termsExist = True

        relPubCount = 0
        relPubPIDCount = 0
        # Get info about related publication metadata
        for field in datasetVersions['data'][0]['metadataBlocks']['citation']['fields']:
            if field['typeName'] == 'publication':
                for value in field['value']:
                    relPubCount += 1
                    try:
                        publicationIDType = value['publicationIDType']
                        publicationIDNumber = value['publicationIDNumber']
                        relPubPIDCount += 1
                    except KeyError:
                        relPubPIDCount = relPubPIDCount
            else:
                relPubCount = relPubCount
        
        # Show metadatablocks whose fields are used by the dataset
        usedMetadataBlocks = []
        for repositoryMetadataBlockName in repositoryMetadataBlockNames:
            try:
                fieldCount = len(datasetVersions['data'][0]['metadataBlocks'][repositoryMetadataBlockName]['fields'])
                if fieldCount > 0:
                    usedMetadataBlocks.append(repositoryMetadataBlockName)
            except KeyError:
                usedMetadataBlocks = usedMetadataBlocks
        if len(usedMetadataBlocks) == 0:
            usedMetadataBlocks = ''
        else:
            usedMetadataBlocks = list_to_string(usedMetadataBlocks)
        
        # Get number of files
        numberOfFiles = len(datasetVersions['data'][0]['files'])

        # Get file info
        noFileDescriptionCount = 0
        contentType = []
        uningestedTabFilesCount = 0
        restrictedFilesCount = 0
        for file in datasetVersions['data'][0]['files']:            
            if 'description' in file:
                noFileDescriptionCount = noFileDescriptionCount
            else:
                noFileDescriptionCount += 1
            contentType.append(file['dataFile']['contentType'])
            if file['restricted'] == True:
                restrictedFilesCount += 1
            if file['dataFile']['contentType'] in uningestedFileTypes:
                uningestedTabFilesCount += 1

        if len(contentType) == 0:
            uniqueContentTypes = 'NA'
        else:
            uniqueContentTypes = list_to_string(list(set(contentType)))

        # Create dictionary
        newRow = {'datasetPID': datasetPID,
                  'datasetPIDUrl' : datasetPID.replace('doi:', 'https://doi.org/'),
                  'numberOfVersions': len(datasetVersions['data']),
                  'publicationDate': string_to_datetime(datasetVersions['data'][firstVersion]['releaseTime']),
                  'latestReleaseDate': string_to_datetime(datasetVersions['data'][0]['releaseTime']),
                  'ageOfDataset(Days)': ageOfDataset,
                  'ageOfLastUpdate(Days)': ageOfLastUpdate,
                  'descriptionLenth': descriptionLength,
                  'termsExist': termsExist,
                  'relPubCount': relPubCount,
                  'relPubPIDCount': relPubPIDCount,
                  'usedMetadataBlocks': usedMetadataBlocks,
                  'numberOfFiles': numberOfFiles,
                  'noFileDescriptionCount': noFileDescriptionCount,
                  'uniqueContentTypes': uniqueContentTypes,
                  'uningestedTabFilesCount': uningestedTabFilesCount,
                  'restrictedFilesCount': restrictedFilesCount
                 }
        rowList.append(dict(newRow))
        datasetCount += 1
        print('%s of %s (%s)' % (datasetCount, len(datasetPIDs), datasetPID))
        
if len(datasetPIDs) != datasetCount:
    print('The metadata of %s dataset(s) could not be retrieved' % (len(datasetPIDs) - datasetCount))


1 of 65 (doi:10.7910/DVN/FZGI5D)
2 of 65 (doi:10.7910/DVN/GWTPTK)
3 of 65 (doi:10.7910/DVN/7ASMDM)
4 of 65 (doi:10.7910/DVN/13KIPQ)
5 of 65 (doi:10.7910/DVN/LSPNAO)
6 of 65 (doi:10.7910/DVN/DBZ7IV)
7 of 65 (doi:10.7910/DVN/TZS58I)
8 of 65 (doi:10.7910/DVN/RZCGED)
9 of 65 (doi:10.7910/DVN/1HTKPJ)
10 of 65 (doi:10.7910/DVN/EHQAY8)
11 of 65 (doi:10.7910/DVN/PPOSWW)
12 of 65 (doi:10.7910/DVN/PWC5EY)
13 of 65 (doi:10.7910/DVN/MHW4OZ)
14 of 65 (doi:10.7910/DVN/YODXEY)
15 of 65 (doi:10.7910/DVN/KCBEON)
16 of 65 (doi:10.7910/DVN/ID2SXK)
17 of 65 (doi:10.7910/DVN/QJKF5N)
18 of 65 (doi:10.7910/DVN/DMY8W9)
19 of 65 (doi:10.7910/DVN/0K3W7C)
20 of 65 (doi:10.7910/DVN/BOG3ZF)
21 of 65 (doi:10.7910/DVN/SBJTNU)
22 of 65 (doi:10.7910/DVN/MIYZYQ)
23 of 65 (doi:10.7910/DVN/XAA5TV)
24 of 65 (doi:10.7910/DVN/IFSGQF)
25 of 65 (doi:10.7910/DVN/COVGMO)
26 of 65 (doi:10.7910/DVN/LXKSIR)
27 of 65 (doi:10.7910/DVN/DBN1UD)
28 of 65 (doi:10.7910/DVN/EKFJYE)
29 of 65 (doi:10.7910/DVN/7UFTV6)
30 of 65 (doi:10.7910/D

In [22]:
df = pd.DataFrame(rowList)               
df

,datasetPID,datasetPIDUrl,numberOfVersions,publicationDate,latestReleaseDate,ageOfDataset(Days),ageOfLastUpdate(Days),descriptionLenth,termsExist,relPubCount,relPubPIDCount,usedMetadataBlocks,numberOfFiles,noFileDescriptionCount,uniqueContentTypes,uningestedTabFilesCount,restrictedFilesCount
0,doi:10.7910/DVN/FZGI5D,https://doi.org/10.7910/DVN/FZGI5D,10,2019-01-29 03:59:21+00:00,2020-06-10 07:41:47+00:00,637,139,194,True,0,0,"citation,geospatial",6,6,text/tab-separated-values,0,2
1,doi:10.7910/DVN/GWTPTK,https://doi.org/10.7910/DVN/GWTPTK,6,2019-04-04 03:27:29+00:00,2020-10-13 07:07:45+00:00,572,14,1576,True,0,0,"citation,geospatial",9,3,"application/msword,application/vnd.openxmlform...",5,0
2,doi:10.7910/DVN/7ASMDM,https://doi.org/10.7910/DVN/7ASMDM,8,2019-01-08 06:15:27+00:00,2020-06-10 05:36:54+00:00,658,139,823,True,1,1,citation,2,1,"application/pdf,text/tab-separated-values",0,0
3,doi:10.7910/DVN/13KIPQ,https://doi.org/10.7910/DVN/13KIPQ,6,2018-12-04 01:32:24+00:00,2020-06-10 05:42:55+00:00,693,139,514,True,0,0,"citation,geospatial",5,3,application/vnd.openxmlformats-officedocument....,4,5
4,doi:10.7910/DVN/LSPNAO,https://doi.org/10.7910/DVN/LSPNAO,4,2020-07-20 06:26:03+00:00,2020-07-22 07:11:32+00:00,99,97,1617,True,1,1,citation,22,22,"text/tab-separated-values,type/x-r-syntax",0,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,doi:10.7910/DVN/LDCXIZ,https://doi.org/10.7910/DVN/LDCXIZ,4,2019-07-22 08:53:39+00:00,2020-06-10 02:45:26+00:00,463,139,145,True,1,1,"citation,geospatial",3,1,"application/pdf,application/vnd.openxmlformats...",0,0
61,doi:10.7910/DVN/ZDVCKE,https://doi.org/10.7910/DVN/ZDVCKE,2,2019-07-22 08:34:02+00:00,2019-07-29 02:16:16+00:00,463,456,1725,True,1,1,citation,1,0,application/pdf,0,0
62,doi:10.7910/DVN/9CNTYS,https://doi.org/10.7910/DVN/9CNTYS,4,2016-01-08 10:44:57+00:00,2019-07-08 06:30:33+00:00,1754,477,356,True,0,0,citation,2,2,"application/pdf,image/tiff",0,0
63,doi:10.7910/DVN/1OCLDP,https://doi.org/10.7910/DVN/1OCLDP,4,2016-01-08 10:44:51+00:00,2020-06-03 05:55:42+00:00,1754,146,576,True,0,0,citation,1,0,application/zipped-shapefile,0,0


In [23]:
# Save dataframe to CSV
csvFileName = '%s_datasets.csv' % (dataverseAlias)
df.to_csv(csvFileName)

In [9]:
# Show date of first published dataset
publicationDates = df['publicationDate']
firstPublicationDate = publicationDates.min()

# Show date of most recently published or updated dataset
latestReleaseDates = df['latestReleaseDate']
lastReleaseDate = latestReleaseDates.min()

# Show age of most recently published or updated dataset
ageOfLastUpdates = df['ageOfLastUpdate']
ageOfLastUpdate = ageOfLastUpdates.min()

# Show average age of datasets
ageOfDatasets = df['ageOfDataset(Days)']
averageAge = ageOfDatasets.mean()

# Show average number of dataset versions
numberOfVersions = df['numberOfVersions']
averageNumberOfVersions = numberOfVersions.mean()

# Create list of datasets with fewer than a certain number of characters in their descriptions
lowDescriptionCount = df[df['descriptionLenth'] < 20]
lowDescriptionCount = lowDescriptionCount['datasetPID']


In [10]:
# List count of each unique file format